In [1]:
import dash
from dash import dcc
from dash import html
from dash import dash_table
from dash import Dash, _dash_renderer
from dash import callback
from dash.dependencies import Input, Output, ClientsideFunction
# from jupyter_dash import JupyterDash

import numpy as np
import pandas as pd
import pathlib
import scipy
import plotly.express as px
import plotly.figure_factory as ff
import dash_mantine_components as dmc
_dash_renderer._set_react_version("18.2.0")

In [2]:
champion_ban_pick_rate = pd.read_csv('csv/champion_ban_pick_rate.csv', index_col=0)
champion_winrate = pd.read_csv('csv/champion_winrate.csv', index_col=0)
full_champion_data = pd.read_csv('csv/full_champion_data.csv', index_col=0)
champion_role_winrate = pd.read_csv('csv/champion_role_winrate.csv', index_col=0)
champion_role_pickrate = pd.read_csv('csv/champion_role_pickrate.csv', index_col=0)
game_duration = pd.read_csv('csv/game_duration.csv', index_col=0)
game_time_graph = pd.read_csv('csv/game_time_graph.csv', index_col=0)
winrate_objective = pd.read_csv('csv/winrate_objective.csv', index_col=0)

In [7]:
winrate_objective_color = ["#4B0082",  "#0000FF", "#008080", "#008000", "#8b0000", "#FFA500", "#FF0000", "#808080", "#000000"]
winrate_role_color = ["#4B0082", "#008080", "#8b0000", "#FF0000", "#000000"]
banpickrate_role_color = ["#0000FF", "#008000", "#FFA500", "#808080", "#000000"]


champion_winrate_fig = ff.create_distplot(
    hist_data=[champion_winrate['0']],
    bin_size=1.2,
    group_labels=["Winrate"],
    show_hist=True,
    show_rug=False,
)

champion_winrate_fig.update_layout(
    title = "Distribution of Champion Win Rates",
    title_x=0.5,
    xaxis_title = "Win Rates (%)",
    yaxis_title = "Distribution",
    template="simple_white"
)

champion_banpickrate_fig = ff.create_distplot(
    hist_data=[champion_ban_pick_rate['0']],
    bin_size=2.2,
    group_labels=["Ban/Pick Rate"],
    show_hist=True,
    show_rug=False,
    colors=['red']
)

champion_banpickrate_fig.update_layout(
    title = "Distribution of Champion Ban/Pick Rates",
    title_x=0.5,
    xaxis_title = "Ban/Pick Rates (%)",
    yaxis_title = "Distribution",
    template="simple_white"
)

In [4]:
champion_role_winrate.columns

Index(['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY'], dtype='object')

In [67]:
app = dash.Dash(
    __name__,
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}],
    external_stylesheets=dmc.styles.ALL
)
app.title = "LOL Analysis Project Data Visualization"

server = app.server
app.config.suppress_callback_exceptions = True

app.layout = dmc.MantineProvider(
    forceColorScheme="light",
    children=[
        dmc.Title("League of Legends Data Visualization and Analysis", id='dashboard-title', size="h3", style={"textAlign": "center"}),
        dmc.Divider(variant="solid", size="md", style={"marginTop": "10px", "marginBottom": "10px"}),
        dmc.Grid([
            dmc.GridCol([
                dmc.Title("Winrate by Objectives", id="winrate-objective-title", size="h5", style={"fontWeight": "bold", "fontStyle": "italic", "textDecoration": "underline"}),
                dmc.RadioGroup(
                    id='winrate-objective-radio', 
                    value=winrate_objective.columns[4], 
                    size='sm',
                    style={"flexWrap": "wrap", "width": "100%"},
                    children=
                        dmc.Group(
                            children=[dmc.GridCol(dmc.Radio(i, value=i, color=winrate_objective_color[winrate_objective.columns.get_loc(i) % len(winrate_objective_color)]), span="auto") for i in winrate_objective.columns]
                            ), 
                ),
                dash_table.DataTable(
                    id='winrate-objective-table', 
                    data=winrate_objective.reset_index().to_dict('records'), 
                    style_table={'overflowX': 'auto'},
                    style_cell={ 'textAlign': 'center', "padding": "10px"},
                    style_header={'fontWeight': 'bold','textAlign': 'center'}
                )
            ], span=6),
            dmc.GridCol([
                dmc.LineChart(
                    id='winrate-objective-linechart',
                    h=500,
                    data=winrate_objective.reset_index().to_dict('records'),
                    dataKey="index",
                    series=[
                        {"name": objective, 
                         "color": winrate_objective_color[i % len(winrate_objective_color)],
                        } 
                         for i, objective in enumerate(winrate_objective.columns)
                    ],
                    curveType="linear",
                    tickLine="xy",
                    xAxisLabel="Rank",
                    yAxisLabel="Winrate (%)",
                    xAxisProps={"angle": "30", "dy": 15}
                )
            ], span=6, style={"paddingTop": "120px"})
        ]),

        dmc.Divider(variant="solid", size="md", style={"marginTop": "10px", "marginBottom": "10px"}),
        dmc.Title("Distribution of Win Rates and Ban/Pick Rates", id="winrate-title", size="h5", style={"fontWeight": "bold", "fontStyle": "italic", "textDecoration": "underline"}),
        dmc.Grid([
            dmc.GridCol([
                dcc.Graph(figure=champion_winrate_fig, id="winrate-fig")
            ], span=6),
            dmc.GridCol([
                dcc.Graph(figure=champion_banpickrate_fig, id="banpickrate-fig")
            ], span=6),
        ]),

        dmc.Divider(variant="solid", size="md", style={"marginTop": "10px", "marginBottom": "10px"}),
        dmc.Grid([
            dmc.GridCol([
                dmc.Title("Distribution of Win Rates at Each Lane", id="roles-winrate-title", size="h5", style={"fontWeight": "bold", "fontStyle": "italic", "textDecoration": "underline"}), 
                dmc.RadioGroup(
                    id="roles-winrate-radio",
                    value = champion_role_winrate.columns[0],
                    size = "sm",
                    style={"flexWrap": "wrap", "width": "100%"},
                    children=
                        dmc.Group(
                            children=[dmc.GridCol(dmc.Radio(i, value=i, color=winrate_role_color[champion_role_winrate.columns.get_loc(i) % len(winrate_role_color)]), span="auto") for i in champion_role_winrate.columns]
                        ), 
                ),
                dash_table.DataTable(
                    id='role-winrate-table', 
                    data=champion_role_winrate.round(4).reset_index().to_dict('records'), 
                    page_size = 12,
                    style_table={'overflowX': 'auto'},
                    style_cell={ 'textAlign': 'center', "padding": "10px"},
                    style_header={'fontWeight': 'bold','textAlign': 'center'}
                ), 
            ], span=6),
            dmc.GridCol([
                dcc.Graph(figure={}, id='role-winrate-fig', style={'height': '700px'})
            ], span=6)
        ])
    ]
)

@app.callback(
    Output("winrate-objective-linechart", "series"),
    Input("winrate-objective-radio", "value"),
)
def update_chart(objective):
    return [
        {
            "name": objective, "color": winrate_objective_color[winrate_objective.columns.get_loc(objective) % len(winrate_objective_color)],
        }
    ]

@app.callback(
    Output("role-winrate-fig", "figure"),
    Input("roles-winrate-radio", "value"),
)
def update_graph(col_chosen):
    fig = ff.create_distplot(
        hist_data=[champion_role_winrate[col_chosen].dropna().to_list()],
        bin_size=0.01,
        group_labels=["Winrate in " + col_chosen + " Role"],
        show_hist=True,
        show_rug=False,
        colors=[winrate_objective_color[champion_role_winrate.columns.get_loc(col_chosen) % len(winrate_objective_color)]]
    )
    
    fig.update_layout(
        title = "Distribution of Champion Win Rates in Each Lane",
        title_x=0.5,
        xaxis_title = "Win Rates (%)",
        yaxis_title = "Distribution",
        template="simple_white"
    )

    return fig

if __name__ == "__main__":
    app.run_server(mode="external", debug=True, port=8050, host="127.0.0.1")